In [2]:
# Load libraries
library(tidyverse)
library(cowplot)
library(Matrix.utils)
library(edgeR)
library(Matrix)
library(reshape2)
library(S4Vectors)
library(SingleCellExperiment)
library(pheatmap)
library(apeglm)
library(png)
library(DESeq2)
library(RColorBrewer)
library(data.table)
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)
library(here)
library(tidyverse)
library(viridis)
library(RColorBrewer)
options (future.globals.maxSize = 4000 * 1024^5)

#color scheme
farmer_colors = c("#FF9999",
"#FF6666",
"#FFCC66",
"#FF6699",
"#CC99FF",
"#CC6699",
"#99CCFF",
"#ff9933",
"#FF99FF",
"#FF99CC",
"#99CCCC",
"#66CC99",
"#CC6600",
"#FFCCCC",
"#339966",
"#CCCCCC",
"#CC9999",
"#996666",
"#FFFF99")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.4     ✔ forcats 1.0.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: limma


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:limma’:

    plotMA


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

  

In [ ]:
#loading in all integrated objects
wt.integrated <- readRDS('/global/homes/k/kserrano/Desktop/Integrated_Objects/wt.integrated2.rds')
ctrl.integrated <- readRDS('/global/homes/k/kserrano/Desktop/Integrated_Objects/ctrl.integrated1.rds')
myc.integrated <- readRDS('/global/homes/k/kserrano/Desktop/Integrated_Objects/myc.integrated.rds')
wtnuc <- readRDS("/global/cfs/projectdirs/m342/plant-microbe_shared/rObjects/med_all_april5.RDS")
myc.nuc <- readRDS('/global/cfs/projectdirs/m342/plant-microbe_shared/rObjects/med.RDS')

In [ ]:
# specify that we will perform downstream analysis on the corrected data note that the
# original unmodified data still resides in the 'Spatial' assay
DefaultAssay(wt.integrated) <- "integrated"
DefaultAssay(ctrl.integrated) <- "integrated"
DefaultAssay(myc.integrated) <- "integrated"
DefaultAssay(wt.nuc) <- "integrated"
DefaultAssay(myc.nuc) <- "integrated"

In [ ]:
#Figure 2 panel B code
##First, let's make our list of AM marker genes from Table S2
am.markers <-c( 'A17-----------MTR-6g021805' #MtEF1a,
'A17-----------CCAMK',
'A17-----------MTR-3g109610',
'A17-----------CNGC15C',
'A17-----------DMI1',
'A17-----------ENOD11',
'A17-----------MTR-5g026850',
'A17-----------LYK3',
'A17-----------MTR-7g100110',
'A17-----------MTR-3g093270',
'A17-----------MTR-3g097560',
'A17-----------MTR-1g112940',
'A17-----------MTR-8g038210',
'A17-----------MTR-0041s0030',
'A17-----------MTR-8g095040',
'A17-----------MTR-1g471050',
'A17-----------MTR-3g065980',
'A17-----------MTR-8g068265',
'A17-----------MTR-1g078400',
'A17-----------MTR-1g017910',
'A17-----------MTR-1g109110',
'A17-----------MTR-1g090920',
'A17-----------MTR-8g006790',
'A17-----------MTR-7g116650',
'A17-----------MTR-5g031030',
'A17-----------MTR-1g028600',
'A17-----------MTR-4g104020',
'A17-----------MTR-7g027190',
'A17-----------MTR-1g040500',
'A17-----------MTR-3g118160',
'A17-----------MTR-5g030910',
'A17-----------MTR-3g089125',
'A17-----------MTR-6g027840',
'A17-----------MTR-8g012805',
'A17-----------MTR-8g012835',
'A17-----------MTR-7g068600',
'R.irregularis-RIR-0168300',
'R.irregularis-RIR-2656400',
'R.irregularis-RIR-3143500')
               
#now let's make the dotplot using the list of AM marker genes  
plot00 <- DotPlot(myc.integrated, assay = "SCT", features = am.markers, cols = c("darkblue", "red"), col.min = -3, col.max = 3, dot.min = 0, dot.scale = 10, idents = NULL, group.by = NULL, split.by = NULL, cluster.idents = TRUE, scale = TRUE, scale.by = "radius", scale.min = NA, scale.max = NA) + RotatedAxis()          
plot00

In [ ]:
##code for Figure 2C
#Integrating single nuclei myc and spatial myc datasets
anchors <- FindTransferAnchors(reference = myc.nuc, query = myc.integrated, normalization.method = "SCT")
imputation.assay <- TransferData(anchorset = anchors, refdata = GetAssayData(myc.nuc[['RNA']]),
weight.reduction = myc.integrated[["pca"]], dims = 1:30)
myc.integrated[["imputation"]] <- imputation.assay
DefaultAssay(myc.integrated) <- "imputation"

#for example, these genes were not detected in the spatial assay, but their location can now be predicted based on concurrently expressed genes in the single cell/single nucleus data
p04 <- SpatialPlot(myc.integrated, features = "A17-----------MTR-0041s0030", images = "slice1.2", alpha =0.5) + scale_fill_viridis_c() #Skl1
p05 <- SpatialPlot(myc.integrated, features = "A17-----------DMI1", images = "slice1.2", alpha =0.5) + scale_fill_viridis_c() #DMI1

#lets now save it as svg
ggsave(file="fig2panelc1.svg", plot=p04, width=12, height=10)
ggsave(file="fig2panelc2.svg", plot=p05, width=12, height=10)

In [ ]:
#proceeding to integration for prediction of cell types within spatial clusters
##simplifying cluster identities into broad categories, pre-annotated object!
###beginning of code for Figure 4b!
Idents(wt.nuc) <- "cell_type"
wt.nuc <- RenameIdents(wt.nuc,"Cortex 1"="Cortex")
wt.nuc<- RenameIdents(object = wt.nuc, `Cortex 2` = "Cortex")
wt.nuc <- RenameIdents(object = wt.nuc, `Cortex 3` = "Cortex")
wt.nuc <- RenameIdents(object = wt.nuc, `Cortex 4` = "Cortex")
wt.nuc <- RenameIdents(object = wt.nuc, `Cortex 5` = "Cortex")
wt.nuc <- RenameIdents(object = wt.nuc, `Cortex 6` = "Cortex")
wt.nuc <- RenameIdents(object = wt.nuc, `Cortex 7` = "Cortex")
wt.nuc <- RenameIdents(object = wt.nuc, `Cortex 8` = "Cortex")
wt.nuc<- RenameIdents(object = wt.nuc, `LRP 1` = "LRP")
wt.nuc <- RenameIdents(object = wt.nuc, `LRP 2` = "LRP")
wt.nuc <- RenameIdents(object = wt.nuc, `Stele 1` = "Stele")
wt.nuc <- RenameIdents(object = wt.nuc, `Stele 2` = "Stele")
wt.nuc <- RenameIdents(object = wt.nuc, `Stele 3` = "Stele")
wt.nuc  <- RenameIdents(object = wt.nuc, `Vascular Pericycle 1` = "Vascular Pericycle")
wt.nuc <- RenameIdents(object = wt.nuc, `Vascular Pericycle 2` = "Vascular Pericycle")
wt.nuc$cell_type<- Idents(wt.nuc)
#check metadata
wtnuc[[]]

In [ ]:
#beginning to transfer data from reference (single-nuclei) to query (spatial)
wt.query <- wt.integrated
wt.anchors <- FindTransferAnchors(reference = wt.nuc, reference.assay = "integrated", query = wt.query,
    dims = 1:30, reference.reduction = "pca")

#storing cell type prediction information as metadata into original spatial object
predictions <- TransferData(anchorset = wt.anchors, refdata = wt.nuc$cell_type,
    dims = 1:30)
wt.integrated <- AddMetaData(wt.integrated, metadata = predictions)
wt.integrated <- MapQuery(anchorset = wt.anchors, reference = wt.nuc, query = wt.query,
    refdata = list(celltype = "cell_type"), reference.reduction = "pca", reduction.model = "umap")

In [ ]:
#comparing cell type labels now between reference and query
p1 <- DimPlot(wt.nuc, reduction = "umap", group.by = "cell_type", label = TRUE, label.size = 3,
    repel = TRUE) + NoLegend() + ggtitle("Reference annotations")
p2 <- DimPlot(wt.integrated, reduction = "ref.umap", group.by = "predicted.celltype", label = TRUE,
    label.size = 3, repel = TRUE)  + ggtitle("Query transferred labels")
p1 + p2

In [1]:
#now lets make a barplot of predicted cell types within each cluster
FetchData(wt.integrated, vars = c("predicted.celltype", "seurat_clusters"))
ggplot(wt.integrated@meta.data, aes(x = seurat_clusters, fill = predicted.id)) + geom_bar() +
    theme_classic()

# SVG graphics device
svg("/global/homes/k/kserrano/Desktop/plot2.svg")

# Code of the plot
ggplot(wt@meta.data, aes(x = seurat_clusters, fill = predicted.id)) + geom_bar() +
    theme_classic()

# Close the graphics device
dev.off() 

###end of code for Figure 4b!

ERROR: Error in FetchData(wt.integrated, vars = c("predicted.celltype", "seurat_clusters")): could not find function "FetchData"


In [ ]:
#beginning of differential gene expression analysis for spatial dataset and data for Figure 4a
#first done for all cells, ignoring clustering, changing cluster identities to a single cluster called "all"
wt$ident<- "all"
head(wt[[]])

In [ ]:
# Bring in Seurat object
seurat <- wt

# Extract raw counts and metadata to create SingleCellExperiment object
counts <- seurat@assays$Spatial@counts 

metadata <- seurat@meta.data

# Set up metadata as desired for aggregation and DE analysis
metadata$ident <- factor(seurat$ident)

# Create single cell experiment object
sce <- SingleCellExperiment(assays = list(counts = counts), 
                           colData = metadata)

In [ ]:
#now on to single cell experiment
assays(sce)
dim(counts(sce))
counts(sce)[1:6, 1:6]

In [ ]:
#exploring cellular metadata
dim(colData(sce))
head(colData(sce))

In [ ]:
#preparing single cell dataset for pseudobulk
# Extract unique names of clusters (= levels of cluster_id factor variable)
cluster_names <- levels(colData(sce)$ident)
cluster_names

# Total number of clusters
length(cluster_names)

In [ ]:
# Extract unique names of samples (= levels of sample_id factor variable)
sample_names <- levels(colData(sce)$sample)
sample_names

# Total number of clusters
length(sample_names)

In [ ]:
#Subset metadata to include only the variables you want to aggregate across 
#(here, we want to aggregate by sample and by cluster)
groups <- colData(sce)[, c("ident", "sample")]
head(groups)

In [ ]:
# Aggregate across cluster-sample groups
# transposing row/columns to have cell_ids as row names matching those of groups
aggr_counts <- aggregate.Matrix(t(counts(sce)), 
                                groupings = groups, fun = "sum") 

# Explore output matrix
class(aggr_counts)
dim(aggr_counts)
aggr_counts[1:6, 1:6]

In [ ]:
# Transpose aggregated matrix to have genes as rows and samples as columns
aggr_counts <- t(aggr_counts)
aggr_counts[1:6, 1:6]

In [ ]:
# Understanding tstrsplit()

## Exploring structure of function output (list)
tstrsplit(colnames(aggr_counts), "_") %>% str()

## Comparing the first 10 elements of our input and output strings
head(colnames(aggr_counts), n = 10)
head(tstrsplit(colnames(aggr_counts), "_")[[1]], n = 10)

In [ ]:
# Using which() to look up tstrsplit() output
all_idx <- which(tstrsplit(colnames(aggr_counts), "_")[[1]] == "all")
all_idx

colnames(aggr_counts)[all_idx]
aggr_counts[1:10, all_idx]

In [ ]:
# As a reminder, we stored our cell types in a vector called cluster_names
cluster_names


# Loop over all cell types to extract corresponding counts, and store information in a list

## Initiate empty list
counts_ls <- list()

for (i in 1:length(cluster_names)) {

  ## Extract indexes of columns in the global matrix that match a given cluster
  column_idx <- which(tstrsplit(colnames(aggr_counts), "_")[[1]] == cluster_names[i])
  
  ## Store corresponding sub-matrix as one element of a list
  counts_ls[[i]] <- aggr_counts[, column_idx]
  names(counts_ls)[i] <- cluster_names[i]

}

# Explore the different components of the list
str(counts_ls)

In [ ]:
# Reminder: explore structure of metadata
head(colData(sce))

# Extract sample-level variables
metadata <- colData(sce) %>% 
  as.data.frame() %>% 
  dplyr::select(sample, treatment)

dim(metadata)
head(metadata)

# Exclude duplicated rows
metadata <- metadata[!duplicated(metadata), ]

dim(metadata)
head(metadata)

In [ ]:
# Rename rows
rownames(metadata) <- metadata$sample
head(metadata)

In [ ]:
# Number of cells per sample and cluster
t <- table(colData(sce)$sample,
           colData(sce)$ident)

In [ ]:
# Creating metadata list

## Initiate empty list
metadata_ls <- list()

for (i in 1:length(counts_ls)) {
  
    ## Initiate a data frame for cluster i with one row per sample (matching column names in the counts matrix)
    df <- data.frame(ident_sample = colnames(counts_ls[[i]]))
    
    ## Use tstrsplit() to separate cluster (cell type) and sample IDs
    df$ident <- tstrsplit(df$ident_sample, "_")[[1]]
    df$sample  <- tstrsplit(df$ident_sample, "_")[[2]]
    
    
    ## Retrieve cell count information for this cluster from global cell count table
    idx <- which(colnames(t) == unique(df$ident))
    cell_counts <- t[, idx]
    
    ## Remove samples with zero cell contributing to the cluster
    cell_counts <- cell_counts[cell_counts > 0]
    
    ## Match order of cell_counts and sample_ids
    sample_order <- match(df$sample, names(cell_counts))
    cell_counts <- cell_counts[sample_order]
    
    ## Append cell_counts to data frame
    df$cell_count <- cell_counts
    
    
    ## Join data frame (capturing metadata specific to cluster) to generic metadata
    df <- plyr::join(df, metadata, 
                     by = intersect(names(df), names(metadata)))
    
    ## Update rownames of metadata to match colnames of count matrix, as needed later for DE
    rownames(df) <- df$ident_sample
    
    ## Store complete metadata for cluster i in list
    metadata_ls[[i]] <- df
    names(metadata_ls)[i] <- unique(df$ident)

}

# Explore the different components of the list
str(metadata_ls)

In [ ]:
#now let's actually look at differential expression!!
# Select cell type of interest
cluster_names

# Double-check that both lists have same names
all(names(counts_ls) == names(metadata_ls))
idx <- which(names(counts_ls) == "all")
cluster_counts <- counts_ls[[idx]]
cluster_metadata <- metadata_ls[[idx]]

In [ ]:
# Check contents of extracted objects
cluster_counts[1:6, 1:6]
head(cluster_metadata)

# Check matching of matrix columns and metadata rows
all(colnames(cluster_counts) == rownames(cluster_metadata))

In [ ]:
# Create DESeq2 object        
dds <- DESeqDataSetFromMatrix(cluster_counts, 
                              colData = cluster_metadata, 
                              design = ~ treatment)

In [ ]:
# Transform counts for data visualization
rld <- rlog(dds, blind=TRUE)

# Plot PCA
DESeq2::plotPCA(rld, ntop = 500, intgroup = "treatment")

In [ ]:
DESeq2::plotPCA(rld, ntop = 500, intgroup = "cell_count")

In [ ]:
# Extract the rlog matrix from the object and compute pairwise correlation values
rld_mat <- assay(rld)
rld_cor <- cor(rld_mat)

# Plot heatmap
pheatmap(rld_cor, annotation = cluster_metadata[, c("treatment"), drop=F])

In [ ]:
# Run DESeq2 differential expression analysis
dds <- DESeq(dds)

In [ ]:
# Plot dispersion estimates
plotDispEsts(dds)

In [ ]:
# Check the coefficients for the comparison
resultsNames(dds)

# Generate results object
res <- results(dds, 
               name = "treatment_Myc_vs_Ctrl",
               alpha = 0.05)

# Shrink the log2 fold changes to be more appropriate using the apeglm method - should cite [paper]() when using this method
res <- lfcShrink(dds, 
                 coef = "treatment_Myc_vs_Ctrl",
                 res=res,
                 type = "apeglm")

In [ ]:
# Turn the DESeq2 results object into a tibble for use with tidyverse functions
res_tbl <- res %>%
  data.frame() %>%
  rownames_to_column(var = "gene") %>%
  as_tibble() %>%
  arrange(padj)

# Check results output
head(res_tbl)

# Write all results to file
write.csv(res_tbl, "/global/homes/k/kserrano/Desktop/resultsall.csv")

In [ ]:
# Set thresholds
padj_cutoff <- 0.05

# Subset the significant results
sig_res <- dplyr::filter(res_tbl, padj < padj_cutoff) %>%
  dplyr::arrange(padj)

# Check significant genes output
sig_res

# Write significant results to file
write.csv(sig_res, "/global/homes/k/kserrano/Desktop/sig.resultsall.csv")

In [ ]:
# Set thresholds
log2fc_cutoff <- 1.0

# Count significantly up/down genes above threshold
n_sig_up <- dplyr::filter(sig_res, log2FoldChange >= log2fc_cutoff) %>% 
  nrow()
n_sig_dn <- dplyr::filter(sig_res, log2FoldChange <= -log2fc_cutoff) %>% 
  nrow()

n_sig_up
n_sig_down

In [ ]:
# Scatterplot

## Extract normalized counts from dds object
normalized_counts <- counts(dds, normalized = TRUE)

## Extract top 20 DEG from resLFC (make sure to order by padj)
top20_sig_genes <- sig_res %>%
  dplyr::arrange(padj) %>%
  dplyr::pull(gene) %>%
  head(n = 20)

## Extract matching normalized count values from matrix
top20_sig_counts <- normalized_counts[rownames(normalized_counts) %in% top20_sig_genes, ]
top20_sig_counts

## Convert wide matrix to long data frame for ggplot2
top20_sig_df <- data.frame(top20_sig_counts)
top20_sig_df$gene <- rownames(top20_sig_counts)

top20_sig_df <- melt(setDT(top20_sig_df), 
                     id.vars = c("gene"),
                     variable.name = "ident_sample") %>% 
  data.frame()
top20_sig_df

## Replace "." by " " in cluster_sample_id variable (melt() introduced the ".")
top20_sig_df$ident_sample <- gsub("\\X", "", top20_sig_df$ident_sample)
top20_sig_df

##make a new dataframe for dds metadata 
df2 <- as.data.frame(colData(dds))
df2


## Join counts data frame with metadata
top20_sig_df <- plyr::join(top20_sig_df, df2,
                           by = "ident_sample")
top20_sig_df

In [ ]:
## Generate plot
plot1 <- ggplot(top20_sig_df, aes(y = value, x = treatment, col = treatment)) +
  geom_jitter(height = 0, width = 0.15) +
  scale_y_continuous(trans = 'log10') +
  ylab("log10 of normalized expression level") +
  xlab("condition") +
  ggtitle("Top 20 Significant DE Genes") +
  theme(plot.title = element_text(hjust = 0.5)) +
  facet_wrap(~ gene)

svg("top20spatial.svg")

# Code of the plot
plot1

# Close the graphics device
dev.off() 

top20_sig_df

In [ ]:
#Heatmap

## Extract normalized counts for significant genes only
sig_counts <- normalized_counts[rownames(normalized_counts) %in% sig_res$gene, ]

## Set a color-blind friendly palette
heat_colors <- rev(brewer.pal(11, "PuOr"))

## Run pheatmap using the metadata data frame for the annotation
pheatmap(sig_counts, 
         color = heat_colors, 
         cluster_rows = TRUE, 
         show_rownames = FALSE,
         annotation = cluster_metadata[, c("treatment", "ident")], 
         border_color = NA, 
         fontsize = 10, 
         scale = "row", 
         fontsize_row = 10, 
         height = 20) 

In [ ]:
# Volcano plot
res_table_thres <- res_tbl[!is.na(res_tbl$padj), ] %>% 
  mutate(threshold = padj < padj_cutoff & abs(log2FoldChange) >= log2fc_cutoff)
min(log10(res_table_thres$padj))

## Generate plot
ggplot(res_table_thres) +
  geom_point(aes(x = log2FoldChange, y = -log10(padj), colour = threshold)) +
  ggtitle("Volcano plot of myc cluster 0 cells relative to ctrl") +
  xlab("log2 fold change") +
  xlim(-4.5, 12) +
  ylab("-log10 adjusted p-value") +
  scale_y_continuous(limits = c(0, 100)) +
  scale_color_manual(values = c("grey60", "red3")) +
  theme(legend.position = "none",
        plot.title = element_text(size = rel(1.3), hjust = 0.5),
        axis.title = element_text(size = rel(1.15)))     

In [ ]:
#end of differential gene expression analysis for all voxels at once
#moving onto doing this analysis for each individual cluster
#code for figure 4c counts

In [ ]:
# Bring in Seurat object
seurat <- wt

# Extract raw counts and metadata to create SingleCellExperiment object
counts <- seurat@assays$Spatial@counts 

metadata <- seurat@meta.data

# Set up metadata as desired for aggregation and DE analysis
#instead of using ident, we now use the seurat_clusters metadata column
metadata$seurat_clusters <- factor(seurat$seurat_clusters)

# Create single cell experiment object
sce <- SingleCellExperiment(assays = list(counts = counts), 
                           colData = metadata)

In [ ]:
#now on to single cell experiment
assays(sce)
dim(counts(sce))
counts(sce)[1:6, 1:6]

In [ ]:
#exploring cellular metadata
dim(colData(sce))
head(colData(sce))

In [ ]:
#preparing single cell dataset for pseudobulk
# Extract unique names of clusters (= levels of cluster_id factor variable)
cluster_names <- levels(colData(sce)$seurat_clusters)
cluster_names

# Total number of clusters
length(cluster_names)

In [ ]:
# Extract unique names of samples (= levels of sample_id factor variable)
sample_names <- levels(colData(sce)$sample)
sample_names

# Total number of clusters
length(sample_names)

In [ ]:
#Subset metadata to include only the variables you want to aggregate across 
#(here, we want to aggregate by sample and by cluster)
groups <- colData(sce)[, c("seurat_clusters", "sample")]
head(groups)

In [ ]:
# Aggregate across cluster-sample groups
# transposing row/columns to have cell_ids as row names matching those of groups
aggr_counts <- aggregate.Matrix(t(counts(sce)), 
                                groupings = groups, fun = "sum") 

# Explore output matrix
class(aggr_counts)
dim(aggr_counts)
aggr_counts[1:6, 1:6]

In [ ]:
# Transpose aggregated matrix to have genes as rows and samples as columns
aggr_counts <- t(aggr_counts)
aggr_counts[1:6, 1:6]

In [ ]:
# Understanding tstrsplit()

## Exploring structure of function output (list)
tstrsplit(colnames(aggr_counts), "_") %>% str()

## Comparing the first 10 elements of our input and output strings
head(colnames(aggr_counts), n = 10)
head(tstrsplit(colnames(aggr_counts), "_")[[1]], n = 10)

In [ ]:
# Using which() to look up tstrsplit() output
#starting with cluster number 0 here
zero_idx <- which(tstrsplit(colnames(aggr_counts), "_")[[1]] == "0")
zero_idx

colnames(aggr_counts)[zero_idx]
aggr_counts[1:10, zero_idx]

In [ ]:
# As a reminder, we stored our cell types in a vector called cluster_names
cluster_names


# Loop over all cell types to extract corresponding counts, and store information in a list

## Initiate empty list
counts_ls <- list()

for (i in 1:length(cluster_names)) {

  ## Extract indexes of columns in the global matrix that match a given cluster
  column_idx <- which(tstrsplit(colnames(aggr_counts), "_")[[1]] == cluster_names[i])
  
  ## Store corresponding sub-matrix as one element of a list
  counts_ls[[i]] <- aggr_counts[, column_idx]
  names(counts_ls)[i] <- cluster_names[i]

}

# Explore the different components of the list
str(counts_ls)

In [ ]:
# Reminder: explore structure of metadata
head(colData(sce))

# Extract sample-level variables
metadata <- colData(sce) %>% 
  as.data.frame() %>% 
  dplyr::select(sample, treatment)

dim(metadata)
head(metadata)

# Exclude duplicated rows
metadata <- metadata[!duplicated(metadata), ]

dim(metadata)
head(metadata)

In [ ]:
# Rename rows
rownames(metadata) <- metadata$sample
head(metadata)

In [ ]:
# Number of cells per sample and cluster
t <- table(colData(sce)$sample,
           colData(sce)$seurat_clusters)

In [ ]:
# Creating metadata list

## Initiate empty list
metadata_ls <- list()

for (i in 1:length(counts_ls)) {
  
    ## Initiate a data frame for cluster i with one row per sample (matching column names in the counts matrix)
    df <- data.frame(seurat_clusters_sample = colnames(counts_ls[[i]]))
    
    ## Use tstrsplit() to separate cluster (cell type) and sample IDs
    df$seurat_clusters <- tstrsplit(df$seurat_clusters_sample, "_")[[1]]
    df$sample  <- tstrsplit(df$seurat_clusters_sample, "_")[[2]]
    
    
    ## Retrieve cell count information for this cluster from global cell count table
    idx <- which(colnames(t) == unique(df$seurat_clusters))
    cell_counts <- t[, idx]
    
    ## Remove samples with zero cell contributing to the cluster
    cell_counts <- cell_counts[cell_counts > 0]
    
    ## Match order of cell_counts and sample_ids
    sample_order <- match(df$sample, names(cell_counts))
    cell_counts <- cell_counts[sample_order]
    
    ## Append cell_counts to data frame
    df$cell_count <- cell_counts
    
    
    ## Join data frame (capturing metadata specific to cluster) to generic metadata
    df <- plyr::join(df, metadata, 
                     by = intersect(names(df), names(metadata)))
    
    ## Update rownames of metadata to match colnames of count matrix, as needed later for DE
    rownames(df) <- df$seurat_clusters_sample
    
    ## Store complete metadata for cluster i in list
    metadata_ls[[i]] <- df
    names(metadata_ls)[i] <- unique(df$seurat_clusters)

}

# Explore the different components of the list
str(metadata_ls)

In [ ]:
# Select cell type of interest
cluster_names

# Double-check that both lists have same names
all(names(counts_ls) == names(metadata_ls))

In [ ]:
idx <- which(names(counts_ls) == "0")
cluster_counts <- counts_ls[[idx]]
cluster_metadata <- metadata_ls[[idx]]

In [ ]:
# Check contents of extracted objects
cluster_counts[1:6, 1:6]
head(cluster_metadata)

# Check matching of matrix columns and metadata rows
all(colnames(cluster_counts) == rownames(cluster_metadata))

In [ ]:
# Create DESeq2 object        
dds0 <- DESeqDataSetFromMatrix(cluster_counts, 
                              colData = cluster_metadata, 
                              design = ~ treatment)

In [ ]:
# Transform counts for data visualization
rld <- rlog(dds0, blind=TRUE)

# Plot PCA
DESeq2::plotPCA(rld, ntop = 500, intgroup = "treatment")

In [ ]:
DESeq2::plotPCA(rld, ntop = 500, intgroup = "cell_count")

In [ ]:
# Extract the rlog matrix from the object and compute pairwise correlation values
rld_mat <- assay(rld)
rld_cor <- cor(rld_mat)

# Plot heatmap
pheatmap(rld_cor, annotation = cluster_metadata[, c("treatment"), drop=F])

In [ ]:
# Run DESeq2 differential expression analysis
dds <- DESeq(dds0)

In [ ]:
# Plot dispersion estimates
plotDispEsts(dds)

In [ ]:
# Check the coefficients for the comparison
resultsNames(dds)

# Generate results object
res0 <- results(dds, 
               name = "treatment_Myc_vs_Ctrl",
               alpha = 0.05)

# Shrink the log2 fold changes to be more appropriate using the apeglm method - should cite [paper]() when using this method
res0 <- lfcShrink(dds, 
                 coef = "treatment_Myc_vs_Ctrl",
                 res=res0,
                 type = "apeglm")

In [ ]:
# Turn the DESeq2 results object into a tibble for use with tidyverse functions
res_tbl <- res0 %>%
  data.frame() %>%
  rownames_to_column(var = "gene") %>%
  as_tibble() %>%
  arrange(padj)

# Check results output
head(res_tbl)

# Write all results to file
write.csv(res_tbl, "/global/homes/k/kserrano/Desktop/results0.csv")

In [ ]:
# Set thresholds
padj_cutoff <- 0.05

# Subset the significant results
sig_res0 <- dplyr::filter(res_tbl, padj < padj_cutoff) %>%
  dplyr::arrange(padj)

# Check significant genes output
sig_res0

# Write significant results to file
write.csv(sig_res0, "/global/homes/k/kserrano/Desktop/sig.results0.csv")

In [ ]:
# Set thresholds
log2fc_cutoff <- 1.0

# Count significantly up/down genes above threshold
n_sig_up0 <- dplyr::filter(sig_res0, log2FoldChange >= log2fc_cutoff) %>% 
  nrow()
n_sig_dn0 <- dplyr::filter(sig_res0, log2FoldChange <= -log2fc_cutoff) %>% 
  nrow()

n_sig_up0
n_sig_dn0

In [ ]:
# Scatterplot

## Extract normalized counts from dds object
normalized_counts <- counts(dds, normalized = TRUE)

## Extract top 20 DEG from resLFC (make sure to order by padj)
top20_sig_genes_0 <- sig_res0 %>%
  dplyr::arrange(padj) %>%
  dplyr::pull(gene) %>%
  head(n = 20)

## Extract matching normalized count values from matrix
top20_sig_counts_0 <- normalized_counts[rownames(normalized_counts) %in% top20_sig_genes_0, ]
top20_sig_counts_0

## Convert wide matrix to long data frame for ggplot2
top20_sig_df_0 <- data.frame(top20_sig_counts_0)
top20_sig_df_0$gene <- rownames(top20_sig_counts_0)

top20_sig_df_0 <- melt(setDT(top20_sig_df_0), 
                     id.vars = c("gene"),
                     variable.name = "seurat_clusters_sample") %>% 
  data.frame()
top20_sig_df_0

## Replace "." by " " in cluster_sample_id variable (melt() introduced the ".")
top20_sig_df_0$seurat_clusters_sample <- gsub("\\X", "", top20_sig_df_0$seurat_clusters_sample)
top20_sig_df_0

##make a new dataframe for dds metadata 
df2 <- as.data.frame(colData(dds))
df2


## Join counts data frame with metadata
top20_sig_df <- plyr::join(top20_sig_df_0, df2,
                           by = "seurat_clusters_sample")
top20_sig_df

In [ ]:
## Generate plot
plot1 <- ggplot(top20_sig_df_0, aes(y = value, x = "treatment", col = "treatment")) +
  geom_jitter(height = 0, width = 0.15) +
  scale_y_continuous(trans = 'log10') +
  ylab("log10 of normalized expression level") +
  xlab("condition") +
  ggtitle("Top 20 Significant DE Genes") +
  theme(plot.title = element_text(hjust = 0.5)) +
  facet_wrap(~ gene)

svg("top20spatial_0.svg")

# Code of the plot
plot1

# Close the graphics device
dev.off() 

top20_sig_df_0

In [ ]:
#Heatmap

## Extract normalized counts for significant genes only
sig_counts <- normalized_counts[rownames(normalized_counts) %in% sig_res0$gene, ]

## Set a color-blind friendly palette
heat_colors <- rev(brewer.pal(11, "PuOr"))

## Run pheatmap using the metadata data frame for the annotation
pheatmap(sig_counts, 
         color = heat_colors, 
         cluster_rows = TRUE, 
         show_rownames = FALSE,
         annotation = cluster_metadata[, c("treatment", "seurat_clusters")], 
         border_color = NA, 
         fontsize = 10, 
         scale = "row", 
         fontsize_row = 10, 
         height = 20) 

In [ ]:
# Volcano plot
res_table_thres0 <- res_tbl[!is.na(res_tbl$padj), ] %>% 
  mutate(threshold = padj < padj_cutoff & abs(log2FoldChange) >= log2fc_cutoff)
min(log10(res_table_thres0$padj))

## Generate plot
ggplot(res_table_thres0) +
  geom_point(aes(x = log2FoldChange, y = -log10(padj), colour = threshold)) +
  ggtitle("Volcano plot of myc cluster 0 cells relative to ctrl") +
  xlab("log2 fold change") +
  xlim(-4.5, 12) +
  ylab("-log10 adjusted p-value") +
  scale_y_continuous(limits = c(0, 100)) +
  scale_color_manual(values = c("grey60", "red3")) +
  theme(legend.position = "none",
        plot.title = element_text(size = rel(1.3), hjust = 0.5),
        axis.title = element_text(size = rel(1.15)))     

In [ ]:
#trying to do all clusters at once, to avoid doing them individually
get_dds_LRTresults <- function(clustx){
  
  print(clustx) # useful for debugging
  
  # Extract counts matrix and metadata for cluster x
  idx <- which(names(counts_ls) == clustx)
  cluster_counts <- counts_ls[[idx]]
  cluster_metadata <- metadata_ls[[idx]]
  
  # Print error message if sample names do not match
  if ( all(colnames(cluster_counts) != rownames(cluster_metadata)) ) {
    print("ERROR: sample names in counts matrix columns and metadata rows do not match!")
  }
  
  # Run DESeq2
  dds <- DESeqDataSetFromMatrix(cluster_counts, 
                                colData = cluster_metadata, 
                                design = ~ treatment)
  dds_lrt <- DESeq(dds, test = "LRT", reduced = ~ 1)
  
  # Extract results
  res_LRT <- results(dds_lrt)
  
  # Create a tibble for LRT results
  res_LRT_tb <- res_LRT %>%
    data.frame() %>%
    rownames_to_column(var = "gene") %>% 
    as_tibble()
  
  # Save all results
  if (!dir.exists("results")) { dir.create("results") }
  write.csv(res_LRT_tb,
            paste0("results/", clustx, "_LRT_all_genes.csv"),
            quote = FALSE, 
            row.names = FALSE)
  
  # Subset to return genes with padj < 0.05
  sigLRT_genes <- res_LRT_tb %>% 
    filter(padj < 0.05)
  
  # Save significant results
  write.csv(sigLRT_genes,
            paste0("results/", clustx, "_LRT_signif_genes.csv"),
            quote = FALSE, 
            row.names = FALSE)
  
  # Transform counts for data visualization
  rld <- rlog(dds_lrt, blind = TRUE)
  
  # Extract the rlog matrix from the object and compute pairwise correlation values
  rld_mat <- assay(rld)
  rld_cor <- cor(rld_mat)
  
  # Obtain rlog values for those significant genes
  cluster_rlog <- rld_mat[sigLRT_genes$gene, ]
  cluster_meta_sig <- cluster_metadata[which(rownames(cluster_metadata) %in% colnames(cluster_rlog)), ]
  
  save(dds_lrt, res_LRT, sigLRT_genes, 
       file = paste0("results/", clustx, "_all_LRTresults.Rdata"))
  
}

map(cluster_names, get_dds_LRTresults)

In [ ]:
#end of differential gene expression!

In [ ]:
#looking at all rhizophagus gene expression
#loading in mycorrhizal spatial objects individually
B1 <- readRDS("/global/homes/k/kserrano/Desktop/Spatial_Objects/B1.rds")
B2 <- readRDS("/global/homes/k/kserrano/Desktop/Spatial_Objects/B2.rds")
B3 <- readRDS("/global/homes/k/kserrano/Desktop/Spatial_Objects/B3.rds")
A2 <- readRDS("/global/homes/k/kserrano/Desktop/Spatial_Objects/A2.rds")
A3 <- readRDS("/global/homes/k/kserrano/Desktop/Spatial_Objects/A3.rds")
C2 <- readRDS("/global/homes/k/kserrano/Desktop/Spatial_Objects/C2.rds")
C3 <- readRDS("/global/homes/k/kserrano/Desktop/Spatial_Objects/C3.rds")
D2 <- readRDS("/global/homes/k/kserrano/Desktop/Spatial_Objects/D2.rds")
D3 <- readRDS("/global/homes/k/kserrano/Desktop/Spatial_Objects/D3.rds")

#merging into a single object
myc.merge <- merge(B1, y = c(B2, B3, A2, A3, C2, C3, D2, D3), add.cell.ids = c("B1", "B2", "B3", "A2", "A3", "C2", "C3", "D2", "D3"))
myc.merge

#getting a list of all genes expressed in this merged object and their expression values
#figure 5 panel a - getting list to compare to previous datasets
apply(myc.merge@assays$Spatial@data,1,mean) -> gene.expression.myc
sort(gene.expression.myc, decreasing = TRUE) -> gene.expression.myc
as.data.frame(gene.expression.myc) -> mycexp
write.csv(mycexp, "/global/homes/k/kserrano/Desktop/mycexp.csv")

In [ ]:
#figure 5 panel c
#plotting dimplot for a particular gene
sobj <- myc.integrated
#ggplot function
get_reduction <- function(sobj, reduction_name) {
  rd <- Embeddings(sobj, reduction = reduction_name) %>% 
    as_tibble(rownames = "Cell")
}

get_cell_data <- function(sobj, features = NULL, melt = TRUE) {
  md <- as_tibble(sobj@meta.data, rownames = "Cell")
  reducs <- names(sobj@reductions)
  rd <- map(reducs, get_reduction, sobj = sobj) %>% reduce(left_join)
  combined_data <- left_join(md, rd)
  if(!is.null(features)) {
    exp_data <- GetAssayData(sobj[features,], slot = "counts", assay = "Spatial") %>% 
      Matrix::t() %>% 
      as_tibble(rownames = "Cell")
    if(melt) {
      exp_data <- gather(exp_data, "Locus", "Counts", -Cell)
    }
    combined_data <- left_join(combined_data, exp_data)
  }
  return(combined_data)
}

## Plot by expression
features <- c("R.irregularis-RIR-2656400") #RiFTR1
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=0.5) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("RiFTR1.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)

## Plot by expression
features <- c("R.irregularis-RIR-0168300") #RiEF1a
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=0.5) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("RiEF1a.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)

## Plot by expression
features <- c("R.irregularis-RIR-1036900") #Rihypothetical
cell_data <- get_cell_data(sobj, features = features)
ggplot(arrange(cell_data, Counts), aes(x = UMAP_1, y = UMAP_2)) +
  geom_point(aes(color = log1p(Counts)),size=0.5) +
  scale_color_viridis_c(name = "Log(Counts + 1)", option = "plasma") +
  labs(x = "UMAP 1", y = "UMAP 2") +
  facet_wrap("Locus") +
  theme_bw() +
  theme(panel.grid = element_blank(),
        axis.line = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank())

ggsave("Rihyp.pdf", plot=last_plot(), device="pdf", width = 5.5, height = 4, units="in", dpi = 300)